<a href="https://colab.research.google.com/github/Lorddickenstein/FSLRwithNLP/blob/main/Tutorials/CNN_Model_Configurations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Activation, Dense, Flatten, MaxPool2D, Conv2D, Dropout
from keras.metrics import categorical_crossentropy

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import glob
import warnings



In [95]:
def import_data():
    """@doc Get the train datasets"""
    df_train = pd.read_csv(
        '/content/drive/MyDrive/Colab Notebooks/Datasets/Fingerspelling/sign_mnist_train/sign_mnist_train.csv')
    # print(df_train.head())

    x_sets = df_train.drop(columns=['label'])
    y_sets = df_train[['label']]

    x_train = x_sets[:22000]
    y_train = y_sets[:22000]
    print(len(x_train))
    # print(y_train.head())

    x_valid = x_sets[22001:]
    y_valid = y_sets[22001:]
    print(len(x_valid))

    """@doc Get the test datasets"""
    df_test = pd.read_csv(
        '/content/drive/MyDrive/Colab Notebooks/Datasets/Fingerspelling/sign_mnist_test/sign_mnist_test.csv')

    x_test = df_test.drop(columns=['label'])
    y_test = df_test[['label']]

    """Convert to np array"""
    x_train = x_train.to_numpy()
    y_train = y_train.to_numpy()
    x_test = x_test.to_numpy()
    y_test = y_test.to_numpy()
    x_valid = x_valid.to_numpy()
    y_valid = y_valid.to_numpy()

    """Reshape to 28x28"""
    x_train = x_train.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)
    x_valid = x_valid.reshape(-1, 28, 28, 1)
    # print(x_train.shape)

    """Normalize data"""
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_valid = x_valid.astype('float32')
    x_train /= 255
    x_test /= 255
    x_valid /= 255

    """Convert to categorical variables"""
    y_train = keras.utils.to_categorical(y_train, 26)
    y_test = keras.utils.to_categorical(y_test, 26)
    y_valid = keras.utils.to_categorical(y_valid, 26)
    
    return (x_train, y_train, x_valid, y_valid, x_test, y_test)

In [92]:
def create_model(dataset):
  # load data
  (x_train, y_train, x_valid, y_valid, x_test, y_test) = dataset
  print("x_train: {}, x_valid: {}, x_test: {}".format(x_train.ndim, x_valid.ndim, x_test.ndim))


  # Create the Sequential Model
  model = Sequential()

  # Add Layers
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
  model.add(MaxPool2D(pool_size=(2, 2)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPool2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.50))
  model.add(Dense(26, activation='softmax'))

  # Summary of layers
  model.summary()

  model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
  model.fit(x=x_train, y=y_train, validation_data=(x_valid, y_valid), batch_size=15, epochs=30)
  print(model.evaluate(x_test, y_test))
  return model

In [ ]:
def save_model(model, name):
  model.save(name)

In [ ]:
def find_match(x):
  classes = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E',
           5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: ' ',
           10: 'K', 11: 'L',12: 'M', 13: 'N', 14: 'O',
           15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T',
           20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y',
           25: ' '}
  return classes[x]

In [ ]:
def predict_image(src_img, model_name):
  model = load_model(model_name)
  prediction = model.predict(src_img)
  class_x = np.argmax(prediction, axis=1)
  return find_match(class_x[0])

In [87]:
path = "/content/drive/MyDrive/Colab Notebooks/Datasets/Test_Images"
file_name = "R2.jpg"
img = cv2.imread(os.path.join(path, file_name))
img = cv2.imread(path, 0)
img = preprocess_image(img)
predict_image(img, )

NameError: ignored

In [96]:
dataset = import_data()

22000
5454


In [97]:
model = create_model(dataset)

x_train: 4, x_valid: 4, x_test: 4
Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_69 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_62 (Dropout)         (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_50 (Dense)   

In [ ]:
model_name = 'test.h5'
path = '/content/drive/MyDrive/Colab Notebooks/CNN Models/' + model_name
save_model(model, path)